In [ ]:
from dotenv import load_dotenv

load_dotenv()

### Input & Output State

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph

model = ChatOpenAI()

In [ ]:
from typing import TypedDict


class ChatMessages(TypedDict):
    question: str
    answer: str
    llm_calls: int

In [ ]:
def call_model(state: ChatMessages):
    question = state["question"]
    llm_calls = state.get("llm_calls", 0)
    state["llm_calls"] = llm_calls + 1
    print("LLM_CALLS:", state["llm_calls"])
    response = model.invoke(input=question)
    state["answer"] = response.content
    return state

In [ ]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [ ]:
graph.invoke(input={"question": "Whats the highest mountain in the world?"})

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


class InputState(TypedDict):
    question: str


class PrivateState(TypedDict):
    llm_calls: int


class OutputState(TypedDict):
    answer: str


class OverallState(InputState, PrivateState, OutputState):
    pass

In [ ]:
workflow = StateGraph(state_schema=OverallState, input_schema=InputState, output_schema=OutputState)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [ ]:
graph.invoke({"question": "Whats the highest mountain in the world?"})

### Add runtime configuration

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables.config import RunnableConfig
from langchain_core.messages import SystemMessage, HumanMessage


def call_model(state: OverallState, config: RunnableConfig):
    language = config["configurable"].get("language", "English")
    system_message_content = "Respond in {}".format(language)
    system_message = SystemMessage(content=system_message_content)
    messages = [system_message, HumanMessage(content=state["question"])]
    response = model.invoke(messages)
    return {"answer": response}

In [ ]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [ ]:
config = {"configurable": {"language": "Spanish"}}
graph.invoke({"question": "What's the highest mountain in the world?"}, config=config)

In [ ]:
config = {"configurable": {"language": "German"}}
graph.invoke({"question": "What's the highest mountain in the world?"}, config=config)

### Runtime Configuration >= LangGraph 0.6.0

In [ ]:
from dataclasses import dataclass

@dataclass
class Context:
    """Per‑run context that replaces `config["configurable"]`.

    Additional attributes can be added later without touching node signatures.
    """
    language: str = "English"

In [ ]:
from langgraph.runtime import Runtime

def call_model(state: OverallState, runtime: Runtime[Context]):
    language = runtime.context.language
    system_message_content = "Respond in {}".format(language)
    system_message = SystemMessage(content=system_message_content)
    messages = [system_message, HumanMessage(content=state["question"])]
    response = model.invoke(messages)
    return {"answer": response}

In [ ]:
workflow_ctx = StateGraph(
    state_schema=OverallState,
    context_schema=Context,        # <‑‑ NEW in v0.6
    input_schema=InputState,
    output_schema=OutputState,
)

workflow_ctx.add_edge(START, "agent")
workflow_ctx.add_node("agent", call_model)
workflow_ctx.add_edge("agent", END)

graph_ctx = workflow_ctx.compile()

In [ ]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [ ]:
context=Context(language="Spanish")
graph.invoke({"question": "What's the highest mountain in the world?"}, context=context)

In [ ]:
context=Context(language="German")
graph.invoke({"question": "What's the highest mountain in the world?"}, context=context)